In [5]:
import re

import lyrics as ly
import json
from pathlib import Path

from spacy.lang.fr import French

def tokenize(chaine):
    return [token.text for token in tokenizer(chaine)]

def preclean(chaine):
    chaine = chaine.replace(u"\xa0", " ")
    chaine = chaine.replace(u"\u2009", " ")
    chaine = chaine.replace(u"\u200b", " ")
    chaine = chaine.replace(u"\u200c", " ")
    return chaine.replace(u"\u200d", " ")

def clean_word(word):
    try:
        word = word.strip()
    except:
        print(word)
        return
    word = word.strip(".,;“…’:!”?\"()[]{}«»×*")
    if re.fullmatch(r"((\\x)|(\\u)|(\\n)|(x?\d+)).*", word):
        return
    if '"-"' in word:
        return
    if word == 'à-ç':
        return
    if re.fullmatch('(-"?\w+)|(\w+"?-)', word):
        return
    if re.fullmatch(r"[^A-zÄ-ÿ]+", word):
        return
    if re.fullmatch(r"('+)|(\++)", word):
        return
    return word

def find_neo(songs):
    neologismes = set()

    for song in songs:
        if song.paroles:
            paroles = tokenize(preclean(song.paroles))
            for word in paroles:
                word = clean_word(word)
                if word:
                    if word.lower() not in lexique_ultime:
                        neologismes.add(word)

    return neologismes

def songs_and_neo(artiste):
    if isinstance(artiste, str):
        artiste = ly.Artiste(artiste)
    songs = artiste.songs
    neologismes = find_neo(songs)
    return songs, neologismes

nlp = French()

tokenizer = nlp.tokenizer

lexiques = Path("lexiques").glob("*.json")

dict_lexiques = {
    fic.stem: set(json.load(fic.open(encoding="utf-8")))
    for fic in lexiques
}

lexique_ultime = set.union(*dict_lexiques.values())

import spacy
nlp = spacy.load("fr_core_news_sm")

In [6]:
dict_artistes = {e : songs_and_neo(e) for e in ("Vald", "FlorentPagny", "Kaaris", "MichelSardou", "OrelSan", "Alizée")}


In [7]:
for k, v in dict_artistes.items():
    print(k, v[0], v[1])

Vald {<Song No friends by Vald>, <Song Adulez-moi by Vald>, <Song Psychose by Vald>, <Song Casimir by Vald>, <Song Interlude by Vald>, <Song Keskivonfer by Vald>, <Song Baby Squirt by Vald>, <Song Megadose by Vald>, <Song Dans ta face by Vald>, <Song XEU morceau abandonné feat Sofiane by Vald>, <Song Désaccordé by Vald>, <Song C’est pas nous les méchants by Vald>, <Song Métafion by Vald (Ft. Suikon Blaz AD)>, <Song Seum by Vald>, <Song Wilson by Vald (Ft. I.N.C.H.)>, <Song Foie gras by Vald>, <Song Strip by Vald>, <Song Freestyle Daymolition by Vald>, <Song Aulnay-sous-Bois by Vald>, <Song Vie de cochon by Vald>, <Song Bukkake by Vald>, <Song Triple Steak by Vald>, <Song Gris by Vald>, <Song Freestyle by Vald (Ft. Georgio)>, <Song Kamasutra by Vald>, <Song Que le Trap Français repose en paix by Vald>, <Song Par toutatis by Vald>, <Song Rendez-moi by Vald (Ft. Suikon Blaz AD)>, <Song Paradoxe by Vald>, <Song Teh stylé by Vald>, <Song YAX3 by Vald>, <Song Possédé by Vald>, <Song Ma meill

In [11]:
import pandas as pd
df = pd.DataFrame(dict_artistes).T
df


,0,1
Vald,"{, [Paroles de ""Adulez-moi""]\n\n[Intro]\nJ'esp...","{Atari, haters, Suiki, délinquanc, Booska, Jam..."
FlorentPagny,"{, , , , , , , , , Quando sono solo sogno all'...","{sonríes, monstruo, parecío, sazón, conocen, i..."
Kaaris,"{, , , , , , , , , , , , , , , , , , , , , , ,...",{}
MichelSardou,"{, , Je n'ai pas la mémoire\nDe mes amours\nMo...","{Sotheby, volar, tualo, Zitrono, astrolomanie,..."
OrelSan,"{, , [Intro]\n- Oh excuse-moi de te déranger, ...","{Beretta, punched, Cougie, Droogies, Kantinov,..."
Alizée,"{, , , , , D'où est d'où vient l'homme\nC'est ...","{See', Memos, envoыtant, dreamt, Aquatic, roll..."


In [13]:
for k, v in dict_artistes.items():
    print(f"Aritste : {k} | Nombre de chansons : {len(v[0])} | Nombre de chansons avec paroles : {len([e for e in v[0] if e.paroles])} | Nombre de neologismes : {len(v[1])}")


Aritste : Vald | Nombre de chansons : 219 | Nombre de chansons avec paroles : 112 | Nombre de neologismes : 716
Aritste : FlorentPagny | Nombre de chansons : 204 | Nombre de chansons avec paroles : 87 | Nombre de neologismes : 556
Aritste : Kaaris | Nombre de chansons : 221 | Nombre de chansons avec paroles : 0 | Nombre de neologismes : 0
Aritste : MichelSardou | Nombre de chansons : 301 | Nombre de chansons avec paroles : 147 | Nombre de neologismes : 272
Aritste : OrelSan | Nombre de chansons : 88 | Nombre de chansons avec paroles : 41 | Nombre de neologismes : 430
Aritste : Alizée | Nombre de chansons : 77 | Nombre de chansons avec paroles : 35 | Nombre de neologismes : 155
